## Read File Fasta dan buat dict{id:seq}

In [152]:
# from Bio import SeqIO
# record_dict = SeqIO.parse("pengbio.fasta_2", "fasta")
# print(record_dict.id)

dict_new = {}
dict_n = []
index = []

#david fasta 2 memuat 5 sequence sample
from Bio import SeqIO
for record in SeqIO.parse("pengbio.fasta_2", "fasta"):
    index = record.id.split('.')
    dict_new.update({index[1]: str(record.seq)})
#     dict_n.append({index[1]:str(record.seq)})

In [153]:
dict_new #nilai seq

{'1': 'GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCTCGGGTTGTAAACCACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACCGCATATGTGACTGCTTGGCTAGAGTGCGTCAGAGGGAGGTGGAATTCCACGTGTAGCAGTGAAATGCGT',
 '2': 'GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCTCGGGTTGTAAACTACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAATTCCCCGGGCTTAACCTGGGTACCGCATATGTGACTGCTTGGCTAGAGTGCGTCAGAGGGAGGTGGAATTCCACGTGTAGCAGTGAAATGCGT',
 '3': 'GGGAATTTTGGACAATGGGCGCAAGCCTGATCCAGCTATTCCGCGTGTGGGATGACGGCCCGTCGGGTTGTAAACTACTTTAGTAGAGAACGAAAAGGAGCTATCGAATAAATAGCTCTGCTGACGGTACTCTAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGTGCAGGCGGCTGAGTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACCGCATATGTGACTGCT

In [1]:
sample = {
    '1': 'GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC',
     '3': 'GTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGTCGTGAAC',
     '2': 'CTTTACCCGGAAGAGCGGGACGCTGCCCTGCGCGATTCCAGGCTCCCCACGGG',
     '5': 'CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC',
     '4': 'TGCGAGGGAAGTGAAGTATTTGACCCTTTACCCGGAAGAGCG',
     '6': 'TGACAGTAGATCTCGTCCAGACCCCTAGCTGGTACGTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGT'
}

In [13]:
sample2 ={
    '1' : 'GGCTCCCCC',
    '3' : 'CCCCCCTTAAA',
    '2' : 'AATTCCGTCC',
    '5' : 'CCTTAACGAA',
    '4' : 'AATTGGCCCT',
    '6' : 'TGACATTGAACT'
}

### menghitung rata rata  panjang seq

In [18]:
def meanLength(data): 
    s = 0
    tot = 0
    for x in data:
        s += len(data[x])
        tot += 1
    return s/float(tot)

In [20]:
meanLength(dict2)

6.0

## fungsi mencari overlap antar 2 seq

In [16]:
def getOverlap(left, right):
    for i in range(len(left)):
        if left[i:] == right[:len(left)-i]:
            return left[i:]
    return ''

In [20]:
getOverlap(sample2['2'], sample2['1'])

''

## mencari overlaps antar seq

In [3]:
def getAllOverlaps(reads):
    d = dict()
    for name1, seq1 in reads.items():
        for name2, seq2 in reads.items():
            if name1 == name2:
                continue
            if name1 not in d:
                d[name1] = dict()
            d[name1][name2] = len(getOverlap(seq1, seq2))

    return d

In [4]:
d=getAllOverlaps(sample)

In [14]:
d

{'1': {'2': 1, '3': 0, '4': 0, '5': 1, '6': 29},
 '2': {'1': 13, '3': 1, '4': 0, '5': 21, '6': 0},
 '3': {'1': 0, '2': 1, '4': 0, '5': 1, '6': 0},
 '4': {'1': 1, '2': 17, '3': 1, '5': 2, '6': 0},
 '5': {'1': 39, '2': 1, '3': 0, '4': 0, '6': 14},
 '6': {'1': 0, '2': 0, '3': 43, '4': 1, '5': 0}}

In [5]:
def findFirstRead(overlaps):
    for i in overlaps:
        signifOverlaps = False
        for j in d[i]:
            if d[j][i] > 3:
                signifOverlaps = True
        if not signifOverlaps:
            return i

In [6]:
findFirstRead(d)

'4'

In [7]:
def findKeyForLargestValue(d):
#     return sorted(d.items(), key=lambda x: x[1])[-1][0]
    m = max(d.values())
    for k in d:
        if d[k] == m:
            return k

## Mencari order seq pada dict untuk membentuk seq baru

In [8]:
def findOrder(first, d):
    if max(d[first].values()) < 3:
        return [first]
    else:
        nextRead = findKeyForLargestValue(d[first])
        return [first] + findOrder(nextRead, d)

In [9]:
order=findOrder(findFirstRead(d), d)

In [10]:
order

['4', '2', '5', '1', '6', '3']

## Assembly gen untuk membentuk seq baru dan jumlah gen terpanjang

In [11]:
def assembleGenome(readOrder, reads, overlaps):
    genome = ''
    for readName in readOrder[:-1]:
        rightOverlap = max(x for x in overlaps[readName].values() if x >= 3)
        genome += reads[readName][:-rightOverlap]
    genome += reads[readOrder[-1]]
    return genome

In [12]:
reference = assembleGenome(order, sample ,d)
print (reference)
print ("Reference gen terpanjang",len(reference))


TGCGAGGGAAGTGAAGTATTTGACCCTTTACCCGGAAGAGCGGGACGCTGCCCTGCGCGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGCTGGTACGTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGTCGTGAAC
Reference gen terpanjang 175
